# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [1]:
import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
    ]
adult_dt = (pd.read_csv('../../05_src/data/adult/adult.data', header = None, names = columns)
            .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))


# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [2]:
from sklearn.model_selection import train_test_split

# reate a dataframe `X` that holds the features
X = adult_dt.drop(columns=['income'])

# Create a dataframe `Y` that holds the target data
Y = adult_dt[['income']]

# Obtain the training and testing data sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=42)

# Display the shapes of the resulting data sets to confirm
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"Y_train shape: {Y_train.shape}")
print(f"Y_test shape: {Y_test.shape}")



X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792, 1)
Y_test shape: (9769, 1)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

+ The random state of the splitting function is a way to control the randomness of the data splitting process. It allows to specify a seed or initial state for the random number generator used to shuffle and split the data.
+ It's useful for:
    * Consistency of results, as the use of a fixed random state ensures that experiments can be repeated by others or by you later under the same conditions. This is crucial to verify the robustness of the model and results.
    * Model comparisons, because when testing multiple models or algorithms, having a fixed random state ensures that each model is trained and tested on the same data splits, making performance comparisons fair and unbiased.
    * Bug tracking, because if a problem occurs in the model, being able to reproduce the exact conditions under which the problem occurred makes debugging easier.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline

# Define the feature columns
numerical_features = ['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss', 'hours-per-week']
categorical_features = ['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'native-country']

# Create a Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num_transforms', 
         Pipeline([
             ('imputer', KNNImputer(n_neighbors=7, weights='distance')),
             ('scaler', RobustScaler())
         ]), 
         numerical_features),
        ('cat_transforms', 
         Pipeline([
             ('imputer', SimpleImputer(strategy='most_frequent')),
             ('encoder', OneHotEncoder(handle_unknown='ignore', drop='if_binary'))
         ]), 
         categorical_features)
    ]
)



![](./images/assignment_2__column_transformer_mine.png)

## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled `preprocessing` and assign the Column Transformer from the previous section.
+ Add a step labelled `classifier` and assign a [`RandomForestClassifier()`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [28]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

# Define the model pipeline
pipeline = Pipeline([
    ('preprocessing', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])


![](./images/assignment_2__pipeline_mine.png)

# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [6]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import make_scorer, log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score
import numpy as np
import warnings
warnings.filterwarnings('ignore')

scoring = [
    'neg_log_loss',
    'roc_auc',
    'accuracy',
    'balanced_accuracy'
]

res_simple_dict = cross_validate(pipeline, X_train, Y_train, cv = 5, scoring = scoring, return_train_score=True)
res_simple = pd.DataFrame(res_simple_dict).assign(experiment = 1)
res_simple

,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
0,8.129849,0.183836,-0.357675,-0.081189,0.904384,1.0,0.850625,0.999945,0.774484,0.999887,1
1,8.631899,0.179607,-0.369239,-0.081516,0.901079,1.0,0.850406,1.000000,0.771881,1.000000,1
2,8.590545,0.177043,-0.375988,-0.081469,0.901378,1.0,0.854103,0.999945,0.776017,0.999887,1
3,8.869196,0.189193,-0.356791,-0.082511,0.907250,1.0,0.859807,1.000000,0.782859,1.000000,1
4,8.906405,0.182270,-0.380379,-0.081368,0.902299,1.0,0.856077,1.000000,0.776089,1.000000,1


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [7]:
df_sorted = res_simple.sort_values(by='test_neg_log_loss', ascending=True)
df_sorted


,fit_time,score_time,test_neg_log_loss,train_neg_log_loss,test_roc_auc,train_roc_auc,test_accuracy,train_accuracy,test_balanced_accuracy,train_balanced_accuracy,experiment
4,8.906405,0.182270,-0.380379,-0.081368,0.902299,1.0,0.856077,1.000000,0.776089,1.000000,1
2,8.590545,0.177043,-0.375988,-0.081469,0.901378,1.0,0.854103,0.999945,0.776017,0.999887,1
1,8.631899,0.179607,-0.369239,-0.081516,0.901079,1.0,0.850406,1.000000,0.771881,1.000000,1
0,8.129849,0.183836,-0.357675,-0.081189,0.904384,1.0,0.850625,0.999945,0.774484,0.999887,1
3,8.869196,0.189193,-0.356791,-0.082511,0.907250,1.0,0.859807,1.000000,0.782859,1.000000,1


Calculate the mean of each metric. 

In [8]:
res_simple.mean()

fit_time                   8.625579
score_time                 0.182390
test_neg_log_loss         -0.368014
train_neg_log_loss        -0.081610
test_roc_auc               0.903278
train_roc_auc              1.000000
test_accuracy              0.854204
train_accuracy             0.999978
test_balanced_accuracy     0.776266
train_balanced_accuracy    0.999955
experiment                 1.000000
dtype: float64

Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [9]:
# Fit the pipeline with training data
pipeline.fit(X_train, Y_train)

# Make predictions
Y_pred_proba = pipeline.predict_proba(X_test)

# Calculate metrics
neg_log_loss = log_loss(Y_test, Y_pred_proba)
roc_auc = roc_auc_score(Y_test, Y_pred_proba[:, 1])
accuracy = accuracy_score(Y_test, pipeline.predict(X_test))
balanced_accuracy = balanced_accuracy_score(Y_test, pipeline.predict(X_test))

# Display results as a dictionary
results_dict = {
    'neg_log_loss': neg_log_loss,
    'roc_auc': roc_auc,
    'accuracy': accuracy,
    'balanced_accuracy': balanced_accuracy
}

results_dict

{'neg_log_loss': 0.39726506589398913,
 'roc_auc': 0.8999828704291436,
 'accuracy': 0.8551540587572934,
 'balanced_accuracy': 0.7751631656838177}

# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recording the target variable immediately after data is loaded is convenient because it combines data transformation with loading, simplifies processing and analysis, avoids redundancy, ensures consistency, and improves code readability and maintainability. This approach is a good practice in data preprocessing workflows, ensuring efficiency and clarity in data-driven projects.

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Evaluation of model pipeline |Model pipeline was evaluated correctly.|Model pipeline was not evaluated correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.